## 参考資料
- [詳細確率ロボティクスのgit](https://github.com/ryuichiueda/LNPR_BOOK_CODES)

## コード内容
- 本書の3.3 まで
- 3.4は4章以降の準備なのでここでは未実装

In [1]:
# conect to drive on colab
from google.colab import drive
drive.mount("/content/drive")
dir_path="./drive/MyDrive/ColabNotebooks/ProbabilisticRobotics/"
# dir_path="./"



Mounted at /content/drive


In [2]:
# import matplotlib
# matplotlib.use("nbagg")
# %matplotlib nbagg
import matplotlib.animation as anm
import matplotlib.pyplot as plt
import math 
import matplotlib.patches as patches
import numpy as np
from IPython.display import HTML

pi=np.pi

In [3]:
class World:
  def __init__(self,time_span,time_interval,debug=False):
    self.objects=[]
    self.debug=debug
    self.time_span=time_span
    self.time_interval=time_interval
  def append(self,obj):
    self.objects.append(obj)
  def draw(self):
    fig=plt.figure(figsize=(5,5))
    ax=fig.add_subplot(111)
    ax.set_aspect("equal")
    ax.set_xlim(-5,5)
    ax.set_ylim(-5,5)
    ax.set_xlabel("X",fontsize=20)
    ax.set_ylabel("Y",fontsize=20)
    elems=[]

    if self.debug:
      for i in range(1000): self.one_step(i,elems,ax)# デバッグ時はアニメーションしない
    else:
      self.ani = anm.FuncAnimation(fig,self.one_step,fargs=(elems,ax),
                                   frames=int(self.time_span/self.time_interval)+1,interval=int(self.time_interval*1000),repeat=False)
      plt.close() #animation on colab 用に追加
      return HTML(self.ani.to_jshtml())#animation on colab 用に追加
    # for obj in self.objects: obj.draw(ax)
    # plt.show()

  def one_step(self,i,elems,ax):
    while elems: elems.pop().remove()# 二重描画禁止
    time_str="t= %.2f[s]"%(self.time_interval*i)
    elems.append(ax.text(-4.4,4.5,time_str,fontsize=10))
    for obj in self.objects:
      obj.draw(ax,elems)
      if hasattr(obj,"one_step"): obj.one_step(self.time_interval) # 1 mean delta t 


In [4]:

class IdealRobbot:
  def __init__(self,pose,agent=None,sensor=None,color="black"):
    self.pose=pose
    self.r=0.2#ロボットの大きさ(ロボットを表す円の半径)
    self.color=color
    self.agent=agent
    self.poses=[pose]
    self.sensor=sensor
    
  def draw(self,ax,elems):
    x,y,theta=self.pose
    xn = x + self.r * math.cos(theta)
    yn = y + self.r * math.sin(theta)

    elems += ax.plot([x,xn],[y,yn],color=self.color)# ロボットの向きの線分
    c = patches.Circle(xy=(x,y),radius=self.r,fill=False,color=self.color)#ロボット本体
    elems.append(ax.add_patch(c))

    # 軌跡描画
    self.poses.append(self.pose)
    elems += ax.plot([e[0] for e in self.poses],[e[1] for e in self.poses],linewidth=0.5,color="black")
    
    # ideal camera描画
    if self.sensor and len(self.poses)>1:
      self.sensor.draw(ax,elems,self.poses[-2])# センサを得た時刻の姿勢

  def one_step(self,time_interval):
    if not self.agent: return
    obs=self.sensor.data(self.pose) if self.sensor else None
    nu,omega=self.agent.decision(obs)
    self.pose=self.state_transition(nu,omega,time_interval,self.pose)


  @classmethod
  def state_transition(cls,nu,omega,time,pose):
    theta0=pose[2]
    if math.fabs(omega)<1e-10:
      return pose + np.array([nu*math.cos(theta0),
                              nu*math.sin(theta0),
                              omega])*time
    else:
      return pose + np.array([nu/omega*(math.sin(theta0+omega*time)-math.sin(theta0)),
                              nu/omega*(-math.cos(theta0+omega*time)+math.cos(theta0)),
                              omega*time])


In [5]:
class Agent:
    def __init__(self,nu,omega):
      self.nu=nu
      self.omega=omega
    def decision(self,observation=None):
      return self.nu,self.omega

In [6]:
class Landmark:
  def __init__(self,x,y):
    self.pos=np.array([x,y]).T
    self.id=None
  def draw(self,ax,elems):
    c=ax.scatter(self.pos[0],self.pos[1],s=100,marker="*",label="landmarks",color="orange")
    elems.append(c)
    elems.append(ax.text(self.pos[0],self.pos[1],"id="+str(self.id),fontsize=10))

In [7]:
class Map:
  def __init__(self):
    self.landmarks=[]
  def append_landmark(self,landmark):
    landmark.id=len(self.landmarks)
    self.landmarks.append(landmark)
  def draw(self,ax,elems):
    for lm in self.landmarks: lm.draw(ax,elems)
 

In [8]:
class IdealCamera:
  def __init__(self,env_map,
               distance_range=(1,1),
               direction_range=(-pi,pi)):
    self.map=env_map
    self.lastdata=[]
    self.distance_range=distance_range
    self.direction_range=direction_range

  def visible(self,polarpos):
    if polarpos is None:
      return False
    
    return self.distance_range[0] <= polarpos[0] <= self.distance_range[1]\
            and self.direction_range[0] <= polarpos[1] <= self.direction_range[1]
  
  def data(self,cam_pose):
    observed=[]
    for lm in self.map.landmarks:
      z = self.observation_function(cam_pose,lm.pos)
      if self.visible(z):
        observed.append((z,lm.id))
    self.lastdata=observed
    return observed
  
  def draw(self,ax,elems,cam_pose):
    for lm in self.lastdata:
      x,y,theta=cam_pose
      distance,direction=lm[0][0],lm[0][1]
      lx = x + distance * math.cos(direction + theta)
      ly = y + distance * math.sin(direction + theta)
      elems+=ax.plot([x,lx],[y,ly],color="pink")
      
  @classmethod
  def observation_function(cls,cam_pose,obj_pos):
    diff = obj_pos- cam_pose[0:2]
    phi = math.atan2(diff[1],diff[0]) - cam_pose[2]
    while phi >= pi: phi -= 2*pi
    while phi < -pi: phi += 2*pi
    return np.array([np.hypot(*diff),phi]).T



In [9]:
def deg_to_rad(deg):
  return deg/180*pi

In [10]:
world=World(30,0.1,debug=False)

m=Map()
m.append_landmark(Landmark(2,-2))
m.append_landmark(Landmark(-1,-3))
m.append_landmark(Landmark(3,3))
world.append(m)

straight=Agent(0.2,0)
circle=Agent(0.2,deg_to_rad(10))
robot1=IdealRobbot(np.array([2,3,pi/6]).T,sensor=IdealCamera(m),agent=straight)
robot2=IdealRobbot(np.array([-2,-1,pi/5*6]).T,sensor=IdealCamera(m),agent=circle,color="red")
# robot3=IdealRobbot(np.array([0,0,0]).T,color="blue")# no agentt
world.append(robot1)
world.append(robot2)
# world.append(robot3)

world.draw()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 実装チェック
IdealRobbot.state_transition(0.1,deg_to_rad(10),9,np.array([0,0,0]).T)
IdealRobbot.state_transition(0.1,deg_to_rad(10),18,np.array([0,0,0]).T)

cam=IdealCamera(m)
p=cam.data(robot2.pose)
p

[(array([4.16661716, 0.92102083]), 0),
 (array([1.11539959, 0.53325976]), 1),
 (array([7.70283017, 1.5813995 ]), 2)]